#### Importowanie modułów

In [ ]:
%cd /content/drive/MyDrive/glebokie/dataset/dogs-vs-cats
!rm Dog/11702.jpg

/content/drive/.shortcut-targets-by-id/1M_VE3u0mnp0u2a22HSNvc9oHa5NiI5w9/dataset/dogs-vs-cats
rm: cannot remove 'Dog/11702.jpg': No such file or directory


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

%ls
%cd /content/drive/MyDrive/glebokie/dataset

# !unrar x data.rar

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Cat/  Dog/
/content/drive/.shortcut-targets-by-id/1M_VE3u0mnp0u2a22HSNvc9oHa5NiI5w9/dataset


In [ ]:
# !pip install --upgrade torch

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torchvision import models, transforms, datasets
from torchvision.models import EfficientNet_B0_Weights
from tqdm import tqdm

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#### Zbiory danych, modele, metody TL, hiperparametry

In [ ]:
from torchvision.models import efficientnet_b0, EfficientNet_B0_Weights
from torchvision.models._api import WeightsEnum
from torch.hub import load_state_dict_from_url


# This is needed for efficient net to work for some reason
def get_state_dict(self, *args, **kwargs):
    kwargs.pop("check_hash")
    return load_state_dict_from_url(self.url, *args, **kwargs)
WeightsEnum.get_state_dict = get_state_dict

# efficientnet_b0(weights=EfficientNet_B0_Weights.IMAGENET1K_V1)
# efficientnet_b0(weights="DEFAULT")

DATASETS = [
    #'CIFAR10',
    'dogs-vs-cats',
    'hats-vs-nohats'
]
MODELS = [
    #models.alexnet(pretrained=True),
    models.vgg16(pretrained=True),
    models.resnet50(pretrained=True),
    models.efficientnet_b0(pretrained=True)
]
TL_METHODS = [
    #'no_freezing',
    'classic_tl',
    'fine_tuning',
    #'our_method'
]

# Hyperparmeters
EPOCHS = 15             # The final number of epochs is twice as big. Learning is split into two phases of EPOCHS size.
BATCH_SIZE = 64

#### Utworzenie Data setów i data loaderów

In [ ]:
def create_datasets_and_dataloaders(dataset_name: str, model):

    if isinstance(model, models.AlexNet) or isinstance(model, models.VGG) or isinstance(model, models.ResNet) or isinstance(model,models.EfficientNet):
        base_transforms = [
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ]
    else:
        base_transforms = [
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ]

    train_transform = transforms.Compose(base_transforms)
    test_transform = transforms.Compose(base_transforms)

    preprocessing = [
        # transforms.RandomResizedCrop(224),
        # transforms.RandomHorizontalFlip(0.5),
        # transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1)
    ]

    train_transform.transforms.extend(preprocessing)

    train_set, test_set, train_loader, test_loader = 0, 0, 0, 0

    if dataset_name == 'CIFAR10':
        train_set = datasets.CIFAR10(root='./data', train=True, download=True, transform=train_transform)
        train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True, num_workers = 16)

        classes = train_set.classes
        test_set = datasets.CIFAR10(root='./data', train=False, download=True, transform=test_transform)
        test_loader = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=False, num_workers = 16)
        print("Loaders created")

    elif dataset_name in ['dogs-vs-cats', 'hats-vs-nohats']:
        train_dataset_full = datasets.ImageFolder(root=f'/content/drive/MyDrive/glebokie/dataset/{dataset_name}', transform=train_transform)
        test_dataset_temp = datasets.ImageFolder(root=f'/content/drive/MyDrive/glebokie/dataset/{dataset_name}', transform=test_transform)

        classes = train_dataset_full.classes
        total_size = len(train_dataset_full)
        train_size = int(0.8 * total_size)
        validation_size = total_size - train_size
        train_set, validation_set = torch.utils.data.random_split(train_dataset_full, [train_size, validation_size])

        test_set = torch.utils.data.Subset(test_dataset_temp, [i for i in range(total_size) if i not in train_set.indices])

        train_loader = torch.utils.data.DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True, num_workers = 16)
        test_loader = torch.utils.data.DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=False, num_workers = 16)
        print("Loaders created")


    return classes, train_loader, test_loader

#### Funkcja mrożąca do naszej metody

In [ ]:
def freeze_percentage(model, percentage):
    """Freeze a percentage of the model's parameters based on gradient norms."""

    # criterion =
    # loss =
    # model.zero_grad()
    # loss.backward()

    grad_params = [(param.grad.norm(), param) for param in model.parameters() if param.requires_grad and param.grad is not None]
    print(grad_params)
    grad_params.sort(key=lambda x: x[0])
    num_params_to_freeze = int(percentage * len(grad_params))
    print("Parameters to freeze:", num_params_to_freeze)
    print("Freezing parameters:", len(grad_params[:num_params_to_freeze]))
    for _, param in grad_params[:num_params_to_freeze]:
        param.requires_grad = False
    model.zero_grad()

#### Przetwarzanie modeli do transfer learningu

In [ ]:
def apply_transfer_learning(model, tl_method):
    if tl_method == 'classic_tl' or tl_method == 'fine_tuning':
        for param in model.parameters():
            param.requires_grad = False

        if isinstance(model, models.AlexNet) or isinstance(model, models.VGG) or isinstance(model, models.EfficientNet):
            for param in model.classifier.parameters():
                param.requires_grad = True
        elif isinstance(model, models.ResNet):
            for param in model.fc.parameters():
                param.requires_grad = True

    elif tl_method == 'no_freezing':
        for param in model.parameters():
            param.requires_grad = True

    # elif tl_method == 'our_method':
    #     freeze_percentage(model,90)


def customize_model(model, num_classes, tl_method):
    if isinstance(model, models.AlexNet):
        dropout = 0.5
        model.classifier = nn.Sequential(
            nn.Dropout(p=dropout),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(p=dropout),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )

    elif isinstance(model, models.VGG):
        dropout = 0.5
        model.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(True),
            nn.Dropout(p=dropout),
            nn.Linear(4096, 4096),
            nn.ReLU(True),
            nn.Dropout(p=dropout),
            nn.Linear(4096, num_classes),
        )

    elif isinstance(model, models.ResNet):
        num_ftrs = model.fc.in_features
        model.fc = nn.Linear(num_ftrs, num_classes)

    elif isinstance(model, models.EfficientNet):
        model.classifier[1] = nn.Linear(in_features=1280, out_features=num_classes)

    print("Classifier for model has been changed.")
    apply_transfer_learning(model, tl_method)
    print("Transfer learning method:", tl_method, "applied.")

    return model

#### Ustawienia fine-tuningu

In [ ]:
def adjust_fine_tuning(model, optimizer):
    """Adjust the model and optimizer for fine-tuning."""
    for param in model.parameters():
        param.requires_grad = True
    for param_group in optimizer.param_groups:
        param_group['lr'] = 0.0001

#### Zlicz zamrożone neurony

In [ ]:
def print_number_of_frozen_params(model):
    frozen_params = 0
    unfrozen_params = 0
    for param in model.parameters():
        if param.requires_grad:
            unfrozen_params += param.numel()
        else:
            frozen_params += param.numel()
    print(f"Liczba zamrożonych parametrów: {frozen_params}")
    print(f"Liczba niezamrożonych parametrów: {unfrozen_params}")

#### Wykresy i tabele

In [ ]:
import matplotlib.pyplot as plt
from prettytable import PrettyTable

def plot_and_save_figures(history, dataset, tl_method, model, save_path):
    plt.figure(figsize=(12, 4))
    plt.suptitle(f"{dataset} - {tl_method} - {model.__class__.__name__}")

    # Plotting training and validation loss
    plt.subplot(1, 2, 1)
    plt.plot(history['train_loss'], label='Train Loss')
    plt.plot(history['val_loss'], label='Validation Loss')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    # Plotting training and validation accuracy
    plt.subplot(1, 2, 2)
    plt.plot(history['train_accuracy'], label='Train Accuracy')
    plt.plot(history['val_accuracy'], label='Validation Accuracy')
    plt.title('Training and Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.savefig(save_path, bbox_inches='tight')
    plt.close()

def add_row_to_table_and_save(table, dataset, tl_method, model, history, file_path):
    last_epoch = len(history['train_loss'])
    train_loss = history['train_loss'][last_epoch - 1]
    val_loss = history['val_loss'][last_epoch - 1]
    train_accuracy = history['train_accuracy'][last_epoch - 1]
    val_accuracy = history['val_accuracy'][last_epoch - 1]

    table.add_row([dataset, tl_method, model.__class__.__name__, train_loss, val_loss, train_accuracy, val_accuracy])

    table_text = table.get_string()
    with open(file_path, "w") as file:
        file.write(table_text)


#### Pętla treningowo walidacyjna

In [ ]:
# Import PrettyTable to store training history
from prettytable import PrettyTable
# Set up the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device = torch.device("cpu")
print(f"Using: {device}")

def train_and_evaluate(model, train_loader, test_loader, criterion, optimizer, epochs, tl_method):
    """Train and evaluate the model."""

    history = {
        'train_loss': [],
        'val_loss': [],
        'train_accuracy': [],
        'val_accuracy': []
    }

    print_number_of_frozen_params(model)

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        correct_train, total_train = 0, 0

        with tqdm(train_loader, desc=f"Epoch {epoch + 1}") as tepoch:
            for images, labels in tepoch:
                images, labels = images.to(device), labels.to(device)

                optimizer.zero_grad()
                outputs = model(images)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()

                running_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                total_train += labels.size(0)
                correct_train += (predicted == labels).sum().item()

                tepoch.set_postfix(loss=loss.item())

        train_accuracy = 100 * correct_train / total_train
        val_loss, val_accuracy = evaluate_model(model, test_loader, criterion, running_loss)

        history['train_loss'].append(running_loss / len(train_loader))
        history['train_accuracy'].append(train_accuracy)
        history['val_loss'].append(val_loss)
        history['val_accuracy'].append(val_accuracy)

    if tl_method == 'fine_tuning' or tl_method == 'our_method':
        adjust_fine_tuning(model, optimizer)
        print("Fine tuning applied. Learning rate: ", optimizer.param_groups[0]['lr'])
        tl_method = 'none'
        fine_tuning_history = train_and_evaluate(model, train_loader, test_loader, criterion, optimizer, epochs, tl_method)
        history = {key: history[key] + fine_tuning_history[key] for key in history}

    elif tl_method == 'classic_tl' or tl_method == 'no_freezing':
        print("Second phase started. Learning rate: ", optimizer.param_groups[0]['lr'])
        tl_method = 'none'
        fine_tuning_history = train_and_evaluate(model, train_loader, test_loader, criterion, optimizer, epochs, tl_method)
        history = {key: history[key] + fine_tuning_history[key] for key in history}

    return history


def evaluate_model(model, test_loader, criterion, running_loss):
    """Evaluate the model and adjust learning rate if fine-tuning."""
    model.eval()
    val_loss, correct_val, total_val = 0.0, 0, 0

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    avg_val_loss = val_loss / len(test_loader)
    val_accuracy = 100 * correct_val / total_val

    print(f'Train Loss: {running_loss/len(train_loader)}, '
          f'Validation Loss: {val_loss/len(test_loader)}, '
          f'Accuracy: {100 * correct_val / total_val}%')

    return avg_val_loss, val_accuracy

# Create the table for metrics
table = PrettyTable()
table.field_names = ["Dataset", "Transfer Learning Method", "Model", "Train Loss", "Validation Loss", "Train Accuracy", "Validation Accuracy"]
file_path = "/content/drive/MyDrive/glebokie/dataset/learning_histories_other.txt"


# Main loop
learning_histories = []

for id_dataset, dataset in enumerate(DATASETS):
    print(f"Using the {dataset} dataset.")

    for id_tl_method, tl_method in enumerate(TL_METHODS):
        print(f"Using the {tl_method} method")

        for id_model, model in enumerate(MODELS):
            if (dataset == "dogs-vs-cats" and tl_method == "classic_tl" and model.__class__.__name__ == "VGG")\
            or (dataset == "dogs-vs-cats" and tl_method == "classic_tl" and model.__class__.__name__ == "ResNet"):
                print("Skipping redundancy")
                continue

            classes, train_loader, test_loader = create_datasets_and_dataloaders(dataset, model=model)
            num_classes = len(classes)

            print(f"Using the {model.__class__.__name__} model")
            model = customize_model(model, num_classes, tl_method)
            criterion = nn.CrossEntropyLoss()
            optimizer = optim.Adam(model.parameters(), lr=0.001)
            model.to(device)

            print("Training starts")
            history = train_and_evaluate(model, train_loader, test_loader, criterion, optimizer, EPOCHS, tl_method)
            learning_histories.append(history)

            add_row_to_table_and_save(table, dataset, tl_method, model, history, file_path)
            save_path = f"/content/drive/MyDrive/glebokie/dataset/{dataset}_{tl_method}_{model.__class__.__name__}.png"
            plot_and_save_figures(history, dataset, tl_method, model, save_path)



Using: cuda
Using the dogs-vs-cats dataset.
Using the classic_tl method
Skipping redundancy
Skipping redundancy


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Loaders created
Using the EfficientNet model
Classifier for model has been changed.
Transfer learning method: classic_tl applied.
Training starts
Liczba zamrożonych parametrów: 4007548
Liczba niezamrożonych parametrów: 2562


Epoch 1:  66%|██████▋   | 208/313 [03:54<00:16,  6.42it/s, loss=0.0809]/usr/local/lib/python3.10/dist-packages/PIL/TiffImagePlugin.py:858: UserWarning: Truncated File Read
  warnings.warn(str(msg))
Epoch 1: 100%|██████████| 313/313 [04:34<00:00,  1.14it/s, loss=0.216]


Train Loss: 0.17595551603327925, Validation Loss: 0.0952476175572676, Accuracy: 96.4%


...

Train Loss: 0.0008008160291811852, Validation Loss: 0.010693033263392366, Accuracy: 99.77777777777777%


Epoch 14: 100%|██████████| 85/85 [00:40<00:00,  2.12it/s, loss=9.22e-5]


Train Loss: 0.001357198058421465, Validation Loss: 0.010024058977178769, Accuracy: 99.77777777777777%


Epoch 15: 100%|██████████| 85/85 [00:41<00:00,  2.05it/s, loss=1.06e-5]


Train Loss: 0.0007552334830132381, Validation Loss: 0.010397229292591832, Accuracy: 99.77777777777777%
